# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra coding portion of the project. 

## The event_datafile_new.csv n which we are woring contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Writing Apache Cassandra code in the cells below

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

# To establish connection and begin executing queries, need a session
session = cluster.connect()

In [6]:
# Creating a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS cass_proj 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

In [7]:
# Setting KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cass_proj')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra we model the database tables on the queries we want to run.

### Query 1:  Querying the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
In this query we choose primary key to be sessionId and itemInSession which results into the unique key.The partition key is sessionId and clustering key is itemInSession.

In [8]:
## Creating table song_description
query= "CREATE TABLE IF NOT EXISTS song_description "
query= query + "(sessionId int, itemInSession int,artist text, song_title text, song_length float,PRIMARY KEY(sessionId,itemInSession)) "

try:
    session.execute(query)
except Exception as e:
    print(e)                    

In [9]:
## Inserting data to the table 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_description ( sessionId, itemInSession,artist, song_title, song_length)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9], float(line[5])))
        

In [10]:
## Verify the data was entered into the table and we get our query result
query= "SELECT artist,song_title,song_length FROM song_description WHERE sessionId= 338 and itemInSession=4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### Query 2: Finding only the following: name of artist, song (sorted by itemInSession) and user (first and last name)for userid = 10, sessionid = 182
In this case, we use primary key in the order: userId, sessionId, itemInSession. We design parttion key to be userId and the clustering columns to be sessionId and itemInSession. Selecting itemInSession as clustering column allows ordering as desired.

In [11]:
## Creating table artist_description
query = "CREATE TABLE IF NOT EXISTS artist_description "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text,\
                    PRIMARY KEY (userId, sessionId,itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [12]:
## Inserting data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO artist_description (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1],line[4]))
        

In [13]:
## Verify the data was entered into the table and we get our query result
query= "SELECT artist,song,itemInSession,song, firstName, lastName FROM artist_description\
            WHERE userId = 10 AND sessionId = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", iteminsession=0, song_="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', iteminsession=1, song_='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', iteminsession=2, song_='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', iteminsession=3, song_='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### Query 3: Finding every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
In this last query, we use primary key song,userId in which song is our desired partion key and the userId is used as clutering column.

In [14]:
## Creating table user_description
query = "CREATE TABLE IF NOT EXISTS user_description "
query = query + "(song text, userId INT, firstName text, lastName text,PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [15]:
## Inserting data to the table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_description (song,userId, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1],line[4]))

In [16]:
## Verify the data was entered into the table and we get our query result
query = "SELECT song, firstName, lastName FROM user_description WHERE song='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row)

Row(song='All Hands Against His Own', firstname='Jacqueline', lastname='Lynch')
Row(song='All Hands Against His Own', firstname='Tegan', lastname='Levine')
Row(song='All Hands Against His Own', firstname='Sara', lastname='Johnson')


In [17]:
## Drop the table before closing out the sessions
query = "DROP TABLE song_description"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE artist_description"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE user_description"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Closing the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()